In [25]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
import requests
import pprint
from enum import Enum
import numpy as np

class Attributes(Enum):
    PACE = 'pace'
    SHOT = 'shot'
    PASS = 'pass'
    DRIBBLE = 'dribble'
    DEFENSE = 'defense'
    PHYSICAL = 'physical'
    
class Positions(Enum):
    ATTACKER = 0
    LEFT_WING = 1
    RIGHT_WING = 2
    CENTER_MID = 3
    DEFENSIVE_MID = 4
    CENTER_BACK = 5
    FULL_BACK = 6
    KEEPER = 7

n_pages = 120    


In [2]:
data = []
for i in range(1, n_pages):
    response = requests.get('https://www.easports.com/fifa/ultimate-team/api/fut/item?page={}'.format(i))
    json = response.json()
    data.append(json)

items = [item['items'] for item in data]
items = [item for sublist in items for item in sublist]

In [3]:
players = []
for item in items:
    player = dict()
    if not item['commonName']:
        player.update({'name': item['firstName'] + ' ' + item['lastName']})
    else:
        player.update({'name': item['commonName']})

    all_attributes = []
    for attribute in item['attributes']:
        attributes = dict()
        attribute_name = attribute['name']
        attribute_value = attribute['value']
        attributes['value'] = attribute_value
        if attribute_name == 'fut.attribute.PAC':
            attributes['name'] = Attributes.PACE.name
        elif attribute_name == 'fut.attribute.SHO':
            attributes['name'] = Attributes.SHOT.name
        elif attribute_name == 'fut.attribute.PAS':
            attributes['name'] = Attributes.PASS.name
        elif attribute_name == 'fut.attribute.DRI':
            attributes['name'] = Attributes.DRIBBLE.name
        elif attribute_name == 'fut.attribute.DEF':
            attributes['name'] = Attributes.DEFENSE.name
        elif attribute_name == 'fut.attribute.PHY':
            attributes['name'] = Attributes.PHYSICAL.name
        all_attributes.append(attributes)
    
    player['attributes'] = all_attributes
    player['rating'] = item['rating']
    if item['position'] == 'ST' or item['position'] == 'CF':
        player['position'] = Positions.ATTACKER.value
    elif item['position'] == 'LW' or item['position'] == 'LM':
        player['position'] = Positions.LEFT_WING.value
    elif item['position'] == 'RW' or item['position'] == 'RM':
        player['position'] = Positions.RIGHT_WING.value
    elif item['position'] == 'CM' or item['position'] == 'CAM':
        player['position'] = Positions.CENTER_MID.value
    elif item['position'] == 'CDM':
        player['position'] = Positions.DEFENSIVE_MID.value
    elif item['position'] == 'CB':
        player['position'] = Positions.CENTER_BACK.value
    elif item['position'] == 'LB' or item['position'] == 'RB':
        player['position'] = Positions.FULL_BACK.value
    elif item['position'] == 'GK':
        player['position'] = Positions.KEEPER.value
    players.append(player)
            

    

In [41]:
player_data = []
labels = []
for i in range(len(players)-100):
    data = []
    for attribute in players[i]['attributes']:
        data.append(attribute['value'])
    data.append(players[i].get('position'))
    player_data.append(data)
    labels.append(players[i]['rating'])
player_data = np.asarray(player_data)
player_data = player_data[0:1]
labels = np.asarray(labels)

In [43]:
# model = Sequential()
# model.add(Dense(64, input_dim=7, activation='relu'))
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mse', metrics=['accuracy'], optimizer=sgd)
# model.fit(player_data, labels, epochs=10, batch_size=64)

In [4]:
pp = pprint.PrettyPrinter(indent=2)
for player in players:
    pp.pprint(player)

{ 'attributes': [ {'name': 'PACE', 'value': 95},
                  {'name': 'SHOT', 'value': 96},
                  {'name': 'PASS', 'value': 93},
                  {'name': 'DRIBBLE', 'value': 96},
                  {'name': 'DEFENSE', 'value': 60},
                  {'name': 'PHYSICAL', 'value': 76}],
  'name': 'Pelé',
  'position': 3,
  'rating': 98}
{ 'attributes': [ {'name': 'PACE', 'value': 92},
                  {'name': 'SHOT', 'value': 93},
                  {'name': 'PASS', 'value': 92},
                  {'name': 'DRIBBLE', 'value': 97},
                  {'name': 'DEFENSE', 'value': 40},
                  {'name': 'PHYSICAL', 'value': 76}],
  'name': 'Diego Maradona',
  'position': 3,
  'rating': 97}
{ 'attributes': [ {'name': 'PACE', 'value': 97},
                  {'name': 'SHOT', 'value': 95},
                  {'name': 'PASS', 'value': 81},
                  {'name': 'DRIBBLE', 'value': 95},
                  {'name': 'DEFENSE', 'value': 45},
                  {'name': 